# AI-Powered Inventory Management & Forecasting

This Kaggle-friendly notebook installs dependencies, runs the training pipeline, and surfaces key outputs. Upload the dataset files (`products.csv`, `transactions.csv`) under `/kaggle/input/<dataset-name>/` or enable synthetic data generation.

## 1. Setup

Uncomment Prophet install if the environment does not already provide it.

In [ ]:
# !pip install prophet

## 2. Configure paths

Point to the dataset directory. For Kaggle competitions/datasets use the provided `/kaggle/input/...` path.

In [ ]:
from pathlib import Path
from src import pipeline

DATA_DIR = Path('/kaggle/input/inventory-data')  # change to your dataset folder
OUTPUT_DIR = Path('/kaggle/working/outputs')
MODEL_NAME = 'prophet'
HORIZON_DAYS = 30
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR

## 3. Run pipeline

If you do not have real data, toggle `USE_SYNTHETIC` to `True` to create demo files.

In [ ]:
from scripts.generate_synthetic import main as generate_synthetic

USE_SYNTHETIC = False
if USE_SYNTHETIC:
    generate_synthetic(out_dir=Path('data'))
    DATA_DIR = Path('data')

outputs = pipeline.run_pipeline(
    data_dir=DATA_DIR,
    output_dir=OUTPUT_DIR,
    model_name=MODEL_NAME,
    horizon_days=HORIZON_DAYS,
)
outputs.reorder.head()

## 4. Visualize outputs

Display reorder recommendations and example plots for Canva.

In [ ]:
from IPython.display import Image
outputs.reorder.sort_values('recommended_reorder_qty', ascending=False).head(10)

In [ ]:
for name, path in outputs.plots.items():
    display(Image(filename=path))
print('Files saved to', OUTPUT_DIR)

## 2b. Use your grocery CSV (optional)

Convert a CSV like `Grocery_Inventory_new_v1.csv` into the pipeline format (`products.csv`, `transactions.csv`).

In [ ]:
from pathlib import Path
from src.adapters.grocery_csv_adapter import convert_grocery_csv_to_internal

# If you uploaded Grocery_Inventory_new_v1.csv to working dir, convert it:
USE_GROCERY_CSV = False
GROCERY_CSV_PATH = Path('/kaggle/working/Grocery_Inventory_new_v1.csv')
if USE_GROCERY_CSV and GROCERY_CSV_PATH.exists():
    convert_grocery_csv_to_internal(GROCERY_CSV_PATH, Path('data'), default_lead_time=7)
    DATA_DIR = Path('data')
DATA_DIR

## 5. Inspect perishable-aware recommendations

Sorted by earliest next_order_date and highest waste risk.

In [ ]:
# Show early orders and potential waste
recs = outputs.reorder.copy()
recs_next = recs.dropna(subset=['next_order_date']).sort_values('next_order_date').head(10)
recs_waste = recs.sort_values('waste_estimate', ascending=False).head(10)
recs_next, recs_waste